In [1]:
import urllib.request
import os.path
import pandas as pd
import numpy as np

from nameparser import HumanName
import humanize


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
if not os.path.isfile("2021_Contributions.csv"):
    url = "https://www.nyccfb.info/DataLibrary/2021_Contributions.csv"
    urllib.request.urlretrieve(url, "2021_Contributions.csv")

In [3]:
donations = pd.read_csv("2021_Contributions.csv", header=0, dtype={ 'NAME': str, 'STRNO': str, 'STRNAME': str, 'APARTMENT': str , 'INTZIP': str, 'EXEMPTCD': str})

In [4]:
donations

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE
0,2021,1,2563,P,"Tirschwell, Sara A",H,7,ABC,NaN,NaN,R0000209,3/6/2021,NaN,"""Dombrowski,"", Ray",IND,NaN,NaN,NaN,NaN,Haverford,PA,19041,Turnaround Manager,Alvarez & Marsal,NaN,600 Madison Avenue,New York,NY,2000.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
1,2021,55,2345,P,"Camarena, Rodrigo G",H,6,ABC,NaN,NaN,R0001664,12/17/2020,NaN,"(Dougherty) LoBianco, Kelly",IND,NaN,NaN,NaN,Z,Los Angeles,CA,90068,Chief Program Officer,The HOPE Program,1,Smith Street,Brooklyn,NY,38.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
2,2021,55,2414,P,"Low, Jenny L",H,6,ABC,NaN,NaN,R0000080,9/5/2020,NaN,"(Eng) Lawton, Linnit",IND,NaN,NaN,NaN,K,Brooklyn,NY,11201,Human Resources,Hamilton Madison House,253,South Street,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
3,2021,55,283,P,"Gennaro, James F",M,7,ABC,NaN,NaN,R0002295,3/3/2021,NaN,"(Orlow) Husarsky, Miriam",IND,NaN,NaN,NaN,Q,Richmond Hill,NY,11418,Speech Therapist,NYC Dept of Education,63-55,102nd street,Rego Park,NY,175.0,175,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
4,2021,55,2454,P,"Boghosian Murphy, Leslie",H,6,ABC,NaN,NaN,R0000171,10/19/2020,NaN,"(Zimmerman)Shaich, Carol",IND,NaN,NaN,NaN,M,Ny,NY,10027,Retired,NaN,NaN,NaN,NaN,NaN,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285576,2021,11,2546,P,"Yang, Andrew",H,7,ABC,NaN,NaN,R0029434,1/13/2021,NaN,"Zymaris, Mai",IND,NaN,NaN,NaN,Z,BRAINTREE,MA,2184,Attorney,Morrison & Foerster,428,John Mahar Hwy Unit 207,BRAINTREE,MA,50.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
285577,2021,11,2393,P,"Wiley, Maya D",H,7,ABC,NaN,NaN,R0034934,2/13/2021,NaN,"Zysermann, Noemie",IND,NaN,NaN,NaN,M,New York,NY,10023,Consultant,Self Employed,266,W 73rd st,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
285578,2021,33,1559,P,"Johnson, Corey D",L,3,ABC,NaN,NaN,R0016683,7/11/2019,NaN,"Zyskowska, Celina",IND,NaN,NaN,NaN,Q,Kew Gardens,NY,11415,Accounting,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,250.0,250,0.0,2,16.0,"Oneill, Brendan",NaN,NaN,NaN,Long Island City,NY,11104,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,Labor Manager,NaN,NaN,NaN,N,N,IND
285579,2021,11,1545,P,"Adams, Eric L",L,1,ABC,NaN,NaN,R0003574,6/12/2018,NaN,"Zyskowski, Joseph",IND,NaN,NaN,NaN,Q,Middle Village,NY,11379,Owner,K Construction,65-58,Admiral Avenue,Middle Village,NY,5000.0,250,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN


In [5]:
mayor_candidates_names = donations[donations['OFFICECD'].isin([1, 11])]['RECIPNAME'].unique()
mayor_donations = donations[donations['OFFICECD'].isin([1, 11])].copy()

major_candidates = [
    'Adams, Eric L',
    'Donovan, Shaun',
    'Garcia, Kathryn A',
    'McGuire, Raymond J',
    'Morales, Dianne',
    'Stringer, Scott M',
    'Wiley, Maya D',
    'Yang, Andrew'
]


candidate_last_names = dict(zip(mayor_candidates_names, map(lambda s: HumanName(s).last, mayor_candidates_names)))
candidate_last_names.values()

dict_values(['Tirschwell', 'Wiley', 'Yang', 'Morales', 'Stringer', 'Cullen', 'Garcia', 'McGuire', 'Donovan', 'Menchaca', 'Adams', 'Diaz', 'Pepitone', 'Taylor', 'Sutton', 'Rose', 'Mateo', 'Guimaraes', 'Chang', 'Sliwa', 'Wright', 'Emilien', 'Filipchenko', 'Laurel-Smith', 'Foldenauer', 'Kavovit', 'Francis', 'Prince', 'Nunez', 'Kaplan', 'Oremland', 'Miles', 'Reaves', 'Coenen', 'Bunea', 'Pinto', 'Downs', 'Flores', 'Fitzgerald', 'Krietchman', 'Nampiaparampil', "O'Hagan", 'Seidman', 'Seely'])

In [6]:
len(mayor_donations)
1500/len(mayor_donations) * 100

1.6310552927744253

In [7]:
mayor_donations.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91965 entries, 0 to 285580
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ELECTION    91965 non-null  int64  
 1   OFFICECD    91965 non-null  int64  
 2   RECIPID     91965 non-null  object 
 3   CANCLASS    91965 non-null  object 
 4   RECIPNAME   91965 non-null  object 
 5   COMMITTEE   91965 non-null  object 
 6   FILING      91965 non-null  int64  
 7   SCHEDULE    91965 non-null  object 
 8   PAGENO      0 non-null      float64
 9   SEQUENCENO  0 non-null      float64
 10  REFNO       91965 non-null  object 
 11  DATE        91960 non-null  object 
 12  REFUNDDATE  2557 non-null   object 
 13  NAME        91965 non-null  object 
 14  C_CODE      91965 non-null  object 
 15  STRNO       0 non-null      object 
 16  STRNAME     0 non-null      object 
 17  APARTMENT   0 non-null      object 
 18  BOROUGHCD   91239 non-null  object 
 19  CITY        91869 non-nu

In [8]:
mayor_donations.NAME.value_counts()

Jordan, Tyron               58
Koo, Julia                  53
Johnson, CHRISTOPHER Leo    47
Warren, Roxanne             44
Johnson, Christopher Leo    37
                            ..
Friedes, Harrison            1
Chesimard, Vernetta          1
zevin, alexandra             1
Niemiec, Robert              1
Cheung, Terrence             1
Name: NAME, Length: 60714, dtype: int64

In [9]:
mayor_donations['C_CODE'].unique()

array(['IND', 'PCOMP', 'CORP', 'CAN', 'OTHR', 'PCOMC', 'FAM', 'PCOMZ',
       'EMPO', 'LLC', 'SPO'], dtype=object)

In [10]:
def parse_name(s):
    names = []
    
    human_name = HumanName(s)
    
    names.append(f"{human_name.last}, {human_name.first}")
    
    if human_name.suffix:
        names.append(f"{human_name.last} {human_name.suffix}, {human_name.first}")
    
    if human_name.middle:
        names.append(f"{human_name.last}, {human_name.first} {human_name.middle}")
    
        if len(human_name.middle) > 1:
            names.append(f"{human_name.last}, {human_name.first} {human_name.middle[0]}")
    
    return names


def name_variations_from_file(file):
    variations = []
    
    with open(file, 'r') as f:
        for line in f:
            for variation in parse_name(line):
                variations.append(variation.upper())

    return variations
                
real_estate_names_from_littlesis = name_variations_from_file('./real_estate_names.txt')
worst_landlords = name_variations_from_file('./worst_landlords.txt')

In [11]:
mayor_donations['realestate_littlesis'] = mayor_donations['NAME'].str.upper().isin(real_estate_names_from_littlesis)
mayor_donations['worst_landlord'] = mayor_donations['NAME'].str.upper().isin(worst_landlords)
mayor_donations['works_in_realestate'] = mayor_donations['OCCUPATION'].str.contains(r'(?i).*REAL ESTATE.*')

In [12]:
flagged_donations = mayor_donations[ (mayor_donations['realestate_littlesis'] | mayor_donations['worst_landlord'] |  mayor_donations['works_in_realestate']  | ( mayor_donations['C_CODE'] == "LLC"))]
flagged_donations_major_candidates = flagged_donations[flagged_donations["RECIPNAME"].isin(major_candidates)]

columns = ['ELECTION', 'RECIPID', 'RECIPNAME', 'COMMITTEE', 'NAME', 'AMNT', 'DATE', 'REFUNDDATE', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME', 'C_CODE', 'realestate_littlesis', 'worst_landlord', 'works_in_realestate']

flagged_donations.sort_values(['RECIPNAME', 'DATE'])[columns]

,ELECTION,RECIPID,RECIPNAME,COMMITTEE,NAME,AMNT,DATE,REFUNDDATE,CITY,STATE,ZIP,OCCUPATION,EMPNAME,C_CODE,realestate_littlesis,worst_landlord,works_in_realestate
49376,2021,1545,"Adams, Eric L",L,"Cohen, Adir",5100.0,1/10/2019,NaN,Brooklyn,NY,11230,Real Estate,Renaissance Realty Group,IND,False,False,True
96693,2021,1545,"Adams, Eric L",L,"Gottlieb, Israel",1000.0,1/10/2019,NaN,Lakewood Township,NJ,8701,Real estate investor,Self-Employed,IND,False,False,True
164427,2021,1545,"Adams, Eric L",L,"Matheus, Donald",500.0,1/10/2019,NaN,Brooklyn,NY,11238,Real estate,Albatross group,IND,False,False,True
249109,2021,1545,"Adams, Eric L",L,"Strulovitch, Baila",2000.0,1/10/2019,NaN,Brooklyn,NY,11205,Real Estate,Self Employee,IND,False,False,True
148543,2021,1545,"Adams, Eric L",L,"Levy, Danny",100.0,1/10/2020,NaN,Brooklyn,NY,11230,Real estate,Self-Employed,IND,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6634,2021,2546,"Yang, Andrew",H,"Alpert, Joseph",1000.0,3/9/2021,NaN,Fort Lee,NJ,7024,Real Estate,Self Employed,IND,False,False,True
19718,2021,2546,"Yang, Andrew",H,"Bellini, Arthur",2000.0,3/9/2021,NaN,New York,NY,10012,Real Estate,Black Bear Asset Management,IND,False,False,True
43526,2021,2546,"Yang, Andrew",H,"Chen, Cindy",10.0,3/9/2021,NaN,Brooklyn,NY,11201,Real estate agent,Compass,IND,False,False,True
85071,2021,2546,"Yang, Andrew",H,"Friedman, Joseph",250.0,3/9/2021,NaN,Brooklyn,NY,11210,Real estate,Self Employed,IND,False,False,True


In [13]:
flagged_donations.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_donations.csv')

In [14]:
def to_money(i):
 return "$" + humanize.intcomma(round(i))

candidate_donations = donations[donations["RECIPNAME"].isin(major_candidates) & donations["SCHEDULE"].eq("ABC")]


In [15]:
total_amount_recieved = candidate_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by="AMNT", ascending=False)
total_amount_recieved["AMNT"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved = total_amount_recieved.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})
total_amount_recieved

,Amount
Candidates,
"McGuire, Raymond J","$7,645,510"
"Adams, Eric L","$3,807,982"
"Stringer, Scott M","$2,555,446"
"Donovan, Shaun","$2,320,507"
"Yang, Andrew","$2,174,180"
"Wiley, Maya D","$1,057,497"
"Garcia, Kathryn A","$586,578"
"Morales, Dianne","$562,751"


In [16]:
def largest_donors_per_candidate(donations, limit=5):       
    agg = donations[["RECIPNAME", "AMNT", "NAME"]].groupby(["RECIPNAME", "NAME"]).agg({'AMNT': sum}) # .sort_values(by='AMNT', ascending=False) # .groupby(level=0, group_keys=False).head(20) # .reset_index() # ort_values(by='AMNT')  # .groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(('AMNT', 'sum'), ascending=False))
    return agg['AMNT'].groupby('RECIPNAME', group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(limit).apply(to_money))
    

In [17]:
# Largest Donors
largest_donors_per_candidate(candidate_donations)

RECIPNAME           NAME                
Adams, Eric L       Bensusan, Danny         $12,100
                    Waisman, Shai           $10,200
                    Guzel, Murat            $10,100
                    Boutross, Diana         $10,000
                    Singh, Navdeep          $10,000
Donovan, Shaun      Donovan, Shaun          $19,100
                    Blumenstein, Ruth       $15,300
                    Briganti, Stephen       $10,200
                    Zabar, Eli              $10,200
                    Nolen, Eliot            $10,200
Garcia, Kathryn A   Smith, Daniel            $2,250
                    sando, vicki             $2,135
                    Hughes, Chris            $2,000
                    Berardi, Eugene          $2,000
                    Wilson, Frederick        $2,000
McGuire, Raymond J  Williams-Isom, Anne     $15,300
                    Rubin, Robert           $10,200
                    Tisch, James            $10,200
                    Lut

In [18]:
# Largest Real Est Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=3)

RECIPNAME           NAME              
Adams, Eric L       Boutross, Diana       $10,000
                    tamir, Elliot          $5,100
                    Verrone, Robert        $5,100
Donovan, Shaun      Fisher, Winston        $5,100
                    DeBode, Gary           $5,100
                    olshan, andrea         $5,100
Garcia, Kathryn A   Kalikow, Peter         $2,000
                    MacArthur, Andrew      $2,000
                    Durst, Helena          $2,000
McGuire, Raymond J  Colacino, Michael      $5,100
                    Goldberg, Andrew       $5,100
                    Trulson, Derek         $5,100
Morales, Dianne     Rosenthal, Richard       $500
                    Neveloff, Jay            $400
                    Fried, Daniel            $350
Stringer, Scott M   Klein, John S         $10,100
                    Quinlan, Robert        $7,900
                    Brodsky, Thomas J      $5,100
Wiley, Maya D       Khan, Fazeel           $2,000
           

In [32]:
def top_donors_for(df, head=20):
    top_donors = df.groupby(["NAME"]).agg({"AMNT": 'sum', 'RECIPNAME': lambda series: list(series.unique()) }).sort_values(by="AMNT", ascending=False).head(head).reset_index()
    top_donors.rename(columns={"NAME": "Donor Name", "AMNT": "Amount", "RECIPNAME": "Supported Candidates" }, inplace=True)
    top_donors['Donor Name'] = top_donors['Donor Name'].apply(lambda x: str(HumanName(x)))
    top_donors["Amount"] = top_donors["Amount"].apply(lambda x: f"${humanize.intcomma(round(x))}" )
    top_donors["Supported Candidates"] = top_donors["Supported Candidates"].apply(lambda names: ", ".join(map(candidate_last_names.get, names)))
    return top_donors


all_top_donors = top_donors_for(mayor_donations, head=50)
top_real_estate_donors = top_donors_for(flagged_donations, head=50)


In [31]:
top_real_estate_donors

,Donor Name,Amount,Supported Candidates
0,Edward Minskoff,"$10,200","Diaz,McGuire"
1,John S Klein,"$10,100",Stringer
2,Diana Boutross,"$10,000",Adams
3,Robert Quinlan,"$7,900",Stringer
4,Peter Kalikow,"$7,100","Garcia,McGuire"
5,Wendy Mosler,"$7,100","Stringer,Sutton"
6,Robert Verrone,"$7,100","Adams,Yang"
7,Daniel Brodsky,"$7,100","Stringer,Donovan"
8,Derek Trulson,"$7,100","Adams,McGuire"
9,Richard Ruben,"$7,100","Adams,McGuire"


In [33]:
all_top_donors

,Donor Name,Amount,Supported Candidates
0,Steven Rattner,"$14,200","Stringer, Adams, Donovan, McGuire"
1,Shaun Donovan,"$14,000",Donovan
2,James Dolan,"$12,200","Adams, Donovan, McGuire"
3,Michael Kempner,"$12,100","Donovan, McGuire"
4,Chris Hughes,"$11,100","Stringer, Adams, Donovan, Garcia"
5,Daniel Lowy,"$10,635","Donovan, Wiley, McGuire, Yang"
6,Joycelyn Taylor,"$10,565",Taylor
7,Elizabeth Gilbert,"$10,200","Diaz, Donovan"
8,Edward Minskoff,"$10,200","Diaz, McGuire"
9,Kenneth Chenault,"$10,200",McGuire
